In [1]:
import pandas as pd

In [61]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

## array自定义函数

### Series的apply方法

In [2]:
df=pd.DataFrame({'a':[1,2,3], 'b':[4,5,6]})
df

,a,b
0,1,4
1,2,5
2,3,6


In [3]:
def func(x):
    return x*x

In [4]:
df['a'].apply(func)

0    1
1    4
2    9
Name: a, dtype: int64

In [5]:
# 接收多个参数
def func1(x,n):
    return x+n

In [6]:
df['a'].apply(func1,args=(3,))

0    4
1    5
2    6
Name: a, dtype: int64

### DataFrame的apply方法

In [7]:
# 按列执行
def func2(x):
    return x-1

In [8]:
df.apply(func2)

,a,b
0,0,3
1,1,4
2,2,5


In [9]:
# 按行执行
df.apply(func2,axis=1)

,a,b
0,0,3
1,1,4
2,2,5


In [10]:
# 按每一个元素执行
df.applymap(func2)

,a,b
0,0,3
1,1,4
2,2,5


## 数据分组

### transform分组

In [11]:
gap=pd.read_csv('./data/gapminder.tsv',sep='\t')
gap

,country,continent,year,lifeExp,pop,gdpPercap
0,Afghanistan,Asia,1952,28.801,8425333,779.445314
1,Afghanistan,Asia,1957,30.332,9240934,820.853030
2,Afghanistan,Asia,1962,31.997,10267083,853.100710
3,Afghanistan,Asia,1967,34.020,11537966,836.197138
4,Afghanistan,Asia,1972,36.088,13079460,739.981106
...,...,...,...,...,...,...
1699,Zimbabwe,Africa,1987,62.351,9216418,706.157306
1700,Zimbabwe,Africa,1992,60.377,10704340,693.420786
1701,Zimbabwe,Africa,1997,46.809,11404948,792.449960
1702,Zimbabwe,Africa,2002,39.989,11926563,672.038623


In [12]:
def func3(x):
    return x-x.mean()

In [13]:
gap.groupby('year')['lifeExp'].transform(func3)

0      -20.256620
1      -21.175401
2      -21.612249
3      -21.658290
4      -21.559386
          ...    
1699    -0.861613
1700    -3.783338
1701   -18.205676
1702   -25.705923
1703   -23.520423
Name: lifeExp, Length: 1704, dtype: float64

In [14]:
# 通过transform填充缺失值
tips=pd.read_csv('./data/tips.csv').sample(10,random_state=42)
tips

,total_bill,tip,sex,smoker,day,time,size
24,19.82,3.18,Male,No,Sat,Dinner,2
6,8.77,2.00,Male,No,Sun,Dinner,2
153,24.55,2.00,Male,No,Sun,Dinner,4
211,25.89,5.16,Male,Yes,Sat,Dinner,4
198,13.00,2.00,Female,Yes,Thur,Lunch,2
176,17.89,2.00,Male,Yes,Sun,Dinner,2
192,28.44,2.56,Male,Yes,Thur,Lunch,2
124,12.48,2.52,Female,No,Thur,Lunch,2
9,14.78,3.23,Male,No,Sun,Dinner,2
101,15.38,3.00,Female,Yes,Fri,Dinner,2


In [15]:
tips.groupby('sex').count()

,total_bill,tip,smoker,day,time,size
sex,,,,,,
Female,3,3,3,3,3,3
Male,7,7,7,7,7,7


In [16]:
import numpy as np
tips.iloc[[1,3,4],0]=np.nan
tips.groupby('sex').count()

,total_bill,tip,smoker,day,time,size
sex,,,,,,
Female,2,3,3,3,3,3
Male,5,7,7,7,7,7


In [17]:
def func4(x):
    avg=x.mean()
    return x.fillna(avg)

In [18]:
new_tips=tips.groupby('sex')['total_bill'].transform(func4)
tips['fill']=new_tips
tips

,total_bill,tip,sex,smoker,day,time,size,fill
24,19.82,3.18,Male,No,Sat,Dinner,2,19.820
6,NaN,2.00,Male,No,Sun,Dinner,2,21.096
153,24.55,2.00,Male,No,Sun,Dinner,4,24.550
211,NaN,5.16,Male,Yes,Sat,Dinner,4,21.096
198,NaN,2.00,Female,Yes,Thur,Lunch,2,13.930
176,17.89,2.00,Male,Yes,Sun,Dinner,2,17.890
192,28.44,2.56,Male,Yes,Thur,Lunch,2,28.440
124,12.48,2.52,Female,No,Thur,Lunch,2,12.480
9,14.78,3.23,Male,No,Sun,Dinner,2,14.780
101,15.38,3.00,Female,Yes,Fri,Dinner,2,15.380


### 分组过滤

In [19]:
tips=pd.read_csv('./data/tips.csv').sample(10,random_state=42)
tips['size'].value_counts()

2    8
4    2
Name: size, dtype: int64

In [20]:
tips.groupby('size').filter(lambda x:x['size'].count()>4)

,total_bill,tip,sex,smoker,day,time,size
24,19.82,3.18,Male,No,Sat,Dinner,2
6,8.77,2.00,Male,No,Sun,Dinner,2
198,13.00,2.00,Female,Yes,Thur,Lunch,2
176,17.89,2.00,Male,Yes,Sun,Dinner,2
192,28.44,2.56,Male,Yes,Thur,Lunch,2
124,12.48,2.52,Female,No,Thur,Lunch,2
9,14.78,3.23,Male,No,Sun,Dinner,2
101,15.38,3.00,Female,Yes,Fri,Dinner,2


### DataFrameGroupBy对象

In [21]:
tips=pd.read_csv('./data/tips.csv').sample(10,random_state=42)

In [22]:
group=tips.groupby('sex')
group.groups # 返回的是行编号

{'Female': [198, 124, 101], 'Male': [24, 6, 153, 211, 176, 192, 9]}

In [23]:
group.max()

,total_bill,tip,smoker,day,time,size
sex,,,,,,
Female,15.38,3.00,Yes,Thur,Lunch,2
Male,28.44,5.16,Yes,Thur,Lunch,4


In [24]:
group.get_group('Female')

,total_bill,tip,sex,smoker,day,time,size
198,13.00,2.00,Female,Yes,Thur,Lunch,2
124,12.48,2.52,Female,No,Thur,Lunch,2
101,15.38,3.00,Female,Yes,Fri,Dinner,2


In [25]:
for g in group:
    print(type(g))
    print(g)

<class 'tuple'>
('Female',      total_bill   tip     sex smoker   day    time  size
198       13.00  2.00  Female    Yes  Thur   Lunch     2
124       12.48  2.52  Female     No  Thur   Lunch     2
101       15.38  3.00  Female    Yes   Fri  Dinner     2)
<class 'tuple'>
('Male',      total_bill   tip   sex smoker   day    time  size
24        19.82  3.18  Male     No   Sat  Dinner     2
6          8.77  2.00  Male     No   Sun  Dinner     2
153       24.55  2.00  Male     No   Sun  Dinner     4
211       25.89  5.16  Male    Yes   Sat  Dinner     4
176       17.89  2.00  Male    Yes   Sun  Dinner     2
192       28.44  2.56  Male    Yes  Thur   Lunch     2
9         14.78  3.23  Male     No   Sun  Dinner     2)


### 多个分组

In [26]:
tips.groupby(['sex','time']).mean()

total_bill       tip      size
sex    time                                  
Female Dinner   15.380000  3.000000  2.000000
       Lunch    12.740000  2.260000  2.000000
Male   Dinner   18.616667  2.928333  2.666667
       Lunch    28.440000  2.560000  2.000000

In [27]:
# 不作为行列标签
tips.groupby(['sex','time'], as_index=False).mean()

,sex,time,total_bill,tip,size
0,Female,Dinner,15.380000,3.000000,2.000000
1,Female,Lunch,12.740000,2.260000,2.000000
2,Male,Dinner,18.616667,2.928333,2.666667
3,Male,Lunch,28.440000,2.560000,2.000000


### 数据透视表

In [31]:
store=pd.read_excel('./data/门店信息表.XLSX')
store

,店铺代码,地区编码,渠道大类,商圈等级描述,商圈类别描述,店铺位置,店铺状态,仓储类别,营业员数,仓储面积,店铺等级,店铺时尚度,接受价格等级,店铺类型,装修代数
0,DPX00X,GBL6020,DZ01,流行,中心,1F中岛,A,店外仓,2.0,15,A,2.0,中,1.0,8代
1,DPX002,GBL6020,DZ01,流行,中心,1F中岛,A,店外仓,2.0,17,A,1.0,中,1.0,7代
2,DPX003,GBL6020,DZ01,流行,中心,2F中岛,A,店外仓,2.0,10,B,2.0,中,1.0,7代
3,DPX004,GBL6020,DZ01,流行,新兴,1F边厅,A,店外仓,2.0,13,B,2.0,中,1.0,7代
4,DPX005,GBL6020,DZ01,流行,中心,1F边厅,A,店外仓,3.0,9,B,1.0,中,1.0,7代
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
786,DPJ034,GBL6010,DZ01,流行,新兴,2F边厅,A,店内仓,4.0,20,B,2.0,中,2.0,8代
787,DPJ036,GBL6120,DZ01,流行,中心,1F边厅,A,店外仓,3.0,30,B,2.0,中,2.0,8代
788,DPJ037,GBL6100,DZ01,大众,中心,二楼边厅,A,店内仓,3.0,20,A,2.0,中,2.0,8代
789,DPJ038,GBL6060,DZ01,折扣,郊区,2F边厅,A,店内仓,6.0,50,A,2.0,中,2.0,NaN


In [37]:
store.groupby(['地区编码','店铺等级']).count()['店铺代码'].unstack()

店铺等级,A,B,C,D
地区编码,,,,
GBL6010,6.0,19.0,14.0,1.0
GBL6020,8.0,20.0,10.0,2.0
GBL6030,25.0,27.0,21.0,2.0
GBL6040,9.0,15.0,18.0,4.0
GBL6050,6.0,16.0,13.0,3.0
GBL6060,6.0,14.0,11.0,2.0
GBL6070,23.0,50.0,37.0,4.0
GBL6080,15.0,45.0,9.0,2.0
GBL6090,8.0,22.0,33.0,3.0


In [38]:
# 通过透视表展示
store.pivot_table(index='地区编码',columns='店铺等级',values='店铺代码',aggfunc='count')

店铺等级,A,B,C,D
地区编码,,,,
GBL6010,6.0,19.0,14.0,1.0
GBL6020,8.0,20.0,10.0,2.0
GBL6030,25.0,27.0,21.0,2.0
GBL6040,9.0,15.0,18.0,4.0
GBL6050,6.0,16.0,13.0,3.0
GBL6060,6.0,14.0,11.0,2.0
GBL6070,23.0,50.0,37.0,4.0
GBL6080,15.0,45.0,9.0,2.0
GBL6090,8.0,22.0,33.0,3.0


## 时间类型

In [39]:
from datetime import datetime

In [40]:
t1=datetime.now()
t1

datetime.datetime(2023, 7, 13, 14, 23, 42, 871065)

In [41]:
t2=datetime(2023,8,23)
t2

datetime.datetime(2023, 8, 23, 0, 0)

In [42]:
t2-t1

datetime.timedelta(days=40, seconds=34577, microseconds=128935)

### pandas时间转换

In [48]:
df=pd.read_csv('./data/country_timeseries.csv',parse_dates=['Date'])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 122 entries, 0 to 121
Data columns (total 18 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   Date                 122 non-null    datetime64[ns]
 1   Day                  122 non-null    int64         
 2   Cases_Guinea         93 non-null     float64       
 3   Cases_Liberia        83 non-null     float64       
 4   Cases_SierraLeone    87 non-null     float64       
 5   Cases_Nigeria        38 non-null     float64       
 6   Cases_Senegal        25 non-null     float64       
 7   Cases_UnitedStates   18 non-null     float64       
 8   Cases_Spain          16 non-null     float64       
 9   Cases_Mali           12 non-null     float64       
 10  Deaths_Guinea        92 non-null     float64       
 11  Deaths_Liberia       81 non-null     float64       
 12  Deaths_SierraLeone   87 non-null     float64       
 13  Deaths_Nigeria       38 non-null   

In [51]:
df['Date_pd']=pd.to_datetime(df['Date'])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 122 entries, 0 to 121
Data columns (total 19 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   Date                 122 non-null    datetime64[ns]
 1   Day                  122 non-null    int64         
 2   Cases_Guinea         93 non-null     float64       
 3   Cases_Liberia        83 non-null     float64       
 4   Cases_SierraLeone    87 non-null     float64       
 5   Cases_Nigeria        38 non-null     float64       
 6   Cases_Senegal        25 non-null     float64       
 7   Cases_UnitedStates   18 non-null     float64       
 8   Cases_Spain          16 non-null     float64       
 9   Cases_Mali           12 non-null     float64       
 10  Deaths_Guinea        92 non-null     float64       
 11  Deaths_Liberia       81 non-null     float64       
 12  Deaths_SierraLeone   87 non-null     float64       
 13  Deaths_Nigeria       38 non-null   

### 提取datetime中的各部分数据

In [62]:
dt=pd.to_datetime('2023-07-13')
dt.year
dt.month
dt.day

2023

7

13

In [64]:
df['year']=df['Date_pd'].dt.year
df['month']=df['Date_pd'].dt.month
df['day']=df['Date_pd'].dt.day
df

,Date,Day,Cases_Guinea,Cases_Liberia,Cases_SierraLeone,Cases_Nigeria,Cases_Senegal,Cases_UnitedStates,Cases_Spain,Cases_Mali,...,Deaths_SierraLeone,Deaths_Nigeria,Deaths_Senegal,Deaths_UnitedStates,Deaths_Spain,Deaths_Mali,Date_pd,year,month,day
0,2015-01-05,289,2776.0,NaN,10030.0,NaN,NaN,NaN,NaN,NaN,...,2977.0,NaN,NaN,NaN,NaN,NaN,2015-01-05,2015,1,5
1,2015-01-04,288,2775.0,NaN,9780.0,NaN,NaN,NaN,NaN,NaN,...,2943.0,NaN,NaN,NaN,NaN,NaN,2015-01-04,2015,1,4
2,2015-01-03,287,2769.0,8166.0,9722.0,NaN,NaN,NaN,NaN,NaN,...,2915.0,NaN,NaN,NaN,NaN,NaN,2015-01-03,2015,1,3
3,2015-01-02,286,NaN,8157.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,2015-01-02,2015,1,2
4,2014-12-31,284,2730.0,8115.0,9633.0,NaN,NaN,NaN,NaN,NaN,...,2827.0,NaN,NaN,NaN,NaN,NaN,2014-12-31,2014,12,31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
117,2014-03-27,5,103.0,8.0,6.0,NaN,NaN,NaN,NaN,NaN,...,5.0,NaN,NaN,NaN,NaN,NaN,2014-03-27,2014,3,27
118,2014-03-26,4,86.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,2014-03-26,2014,3,26
119,2014-03-25,3,86.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,2014-03-25,2014,3,25
120,2014-03-24,2,86.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,2014-03-24,2014,3,24


In [66]:
df['Date'].min()

Timestamp('2014-03-22 00:00:00')

In [70]:
df['diff_day']=df['Date']-df['Date'].min()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 122 entries, 0 to 121
Data columns (total 23 columns):
 #   Column               Non-Null Count  Dtype          
---  ------               --------------  -----          
 0   Date                 122 non-null    datetime64[ns] 
 1   Day                  122 non-null    int64          
 2   Cases_Guinea         93 non-null     float64        
 3   Cases_Liberia        83 non-null     float64        
 4   Cases_SierraLeone    87 non-null     float64        
 5   Cases_Nigeria        38 non-null     float64        
 6   Cases_Senegal        25 non-null     float64        
 7   Cases_UnitedStates   18 non-null     float64        
 8   Cases_Spain          16 non-null     float64        
 9   Cases_Mali           12 non-null     float64        
 10  Deaths_Guinea        92 non-null     float64        
 11  Deaths_Liberia       81 non-null     float64        
 12  Deaths_SierraLeone   87 non-null     float64        
 13  Deaths_Nigeria      

### 日期范围

In [77]:
range_date=pd.date_range(start='2014-12-31',end='2015-01-05')
range_date

DatetimeIndex(['2014-12-31', '2015-01-01', '2015-01-02', '2015-01-03',
               '2015-01-04', '2015-01-05'],
              dtype='datetime64[ns]', freq='D')

In [75]:
df_head=df.iloc[:5,:5]
df_head.index=df_head['Date']
df_head

,Date,Day,Cases_Guinea,Cases_Liberia,Cases_SierraLeone
Date,,,,,
2015-01-05,2015-01-05,289,2776.0,NaN,10030.0
2015-01-04,2015-01-04,288,2775.0,NaN,9780.0
2015-01-03,2015-01-03,287,2769.0,8166.0,9722.0
2015-01-02,2015-01-02,286,NaN,8157.0,NaN
2014-12-31,2014-12-31,284,2730.0,8115.0,9633.0


In [78]:
df_head.reindex(range_date)

,Date,Day,Cases_Guinea,Cases_Liberia,Cases_SierraLeone
2014-12-31,2014-12-31,284.0,2730.0,8115.0,9633.0
2015-01-01,NaT,NaN,NaN,NaN,NaN
2015-01-02,2015-01-02,286.0,NaN,8157.0,NaN
2015-01-03,2015-01-03,287.0,2769.0,8166.0,9722.0
2015-01-04,2015-01-04,288.0,2775.0,NaN,9780.0
2015-01-05,2015-01-05,289.0,2776.0,NaN,10030.0
